# 代码部分

In [ ]:
# 数据库连接
import psycopg2
import copy
conn = psycopg2.connect(database="mississippi", user="postgres", password="1997pgsql", host="127.0.0.1", port="5432")
print("Connect database successfully") 
cur=conn.cursor()

In [ ]:
# conn.rollback()

# 根据关键词选出作为路径的way，新建一张表保存

In [ ]:
%%time
cur.execute('drop table if exists qualified_way_nodes')
sql="""
create table qualified_way_nodes as select *
    from way_nodes
 	where way_id in (
		select distinct way_id
 		from way_tags
 		where k='highway' )
"""
cur.execute(sql)
conn.commit()
cur.execute("select count(distinct way_id) as count from qualified_way_nodes")
res=cur.fetchall()[0][0]
print('extract qualified ways done. there are total %d ways'%(res))
# k='cycleway' or k='oneway' or k='highway'or k='bridge' or k='bike' or k='busway' or k='expressway'or k like '%bus%' or k like 'car%' or k='horse' or k='railway' or k='footway' or k='sidewalk' or k='driveway' or k='way' or k='runway' or k='bicycle'

# 计算路径是否可以组成一个连通图

In [ ]:
import random

node_id_list=[]
cur.execute("select node_id from qualified_way_nodes limit 1000")
res=cur.fetchall()
for item in res:
    node_id_list.append(item[0])
seed_node_id=random.choice(node_id_list)

way_id_set=set()
node_id_set=set()
node_id_set.add(seed_node_id)

In [ ]:
%%time
while 1:
    print('number of new node_id to be query:',len(node_id_set))
    sql_str="("
    for node in node_id_set:
        sql_str+=str(node)
        sql_str+=','
    sql_str=sql_str[:-1]
    sql_str+=")"
    node_id_set=set()
    if sql_str==")":
        break
    cur.execute("select distinct way_id from qualified_way_nodes where node_id in"+sql_str)
    res=cur.fetchall()
    new_way_id_set=set()
    for item in res:
        new_way_id_set.add(item[0])
    if new_way_id_set-way_id_set==False:
        break
    candidate_way_id_set=new_way_id_set-way_id_set
    way_id_set=way_id_set|new_way_id_set
    print('the size of way_id_set is:%d'%(len(way_id_set)))
    
    new_node_id_set=set()
    print('number of new way_id to be query:',len(candidate_way_id_set))
    if len(candidate_way_id_set)==0:
        break
    sql_str="("
    for way_id in candidate_way_id_set:
        sql_str+=str(way_id)
        sql_str+=','
    sql_str=sql_str[:-1]
    sql_str+=")"
    cur.execute("select distinct node_id from qualified_way_nodes where way_id in"+sql_str)
    result=cur.fetchall()
    for item in result:
        new_node_id_set.add(item[0])
    node_id_set=new_node_id_set
print(len(way_id_set))

In [ ]:
all_way_id=set()
cur.execute("select way_id from qualified_way_nodes")
res=cur.fetchall()
for item in res:
    all_way_id.add(item[0])
disqualified_way_id=all_way_id-way_id_set
len(disqualified_way_id)

In [ ]:
# 删除那些不连通的way_id
sql_delete="delete from qualified_way_nodes where way_id in ("
for wayid in disqualified_way_id:
    sql_delete+=str(wayid)
    sql_delete+=","
sql_delete=sql_delete[:-1]
sql_delete+=")"
cur.execute(sql_delete)
conn.commit()
cur.execute("select count(distinct way_id) as count from qualified_way_nodes")
res=cur.fetchall()[0][0]
print('filtered qualified ways done. there are total %d ways'%(res))

# 选出作为结点的node，新建一张表来保存

In [ ]:
%%time 
cur.execute('drop table if exists qualified_nodes')
sql="""
create table qualified_nodes as select id,geom 
from nodes 
where nodes.id in (
	select node_id
 	from qualified_way_nodes )
"""
cur.execute(sql)
conn.commit()
cur.execute("select count(id) as count from qualified_nodes")
res=cur.fetchall()[0][0]
print('extact qualified nodes node. there are total %d nodes.'%(res))

# 记录每个node在记录中的出现频率，将用来确定是否作为路径交点

In [ ]:
%%time
cur.execute('drop table if exists node_freq')
#cur.execute('create table node_freq(node_id bigint not null primary key,freq bigint)')
conn.commit()
sql="create table node_freq as select node_id, count(way_id) as freq from qualified_way_nodes group by node_id"
cur.execute(sql)
conn.commit()
print('count node frequency done.')

# 构建路网数据集主要代码

In [ ]:
# 方法定义
from math import radians, cos, sin, asin, sqrt
import time
import random
 
def haversine(lon1, lat1, lon2, lat2): # 经度1，纬度1，经度2，纬度2 （十进制度数）
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # 将十进制度数转化为弧度
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
 
    # haversine公式
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # 地球平均半径，单位为公里
    return c * r * 1000

def calculate_road_length(node_list,node_lonlat_dict): # node_lonlat_dict: key=nodeid,value=[id,lon,lat]
    try:
        length=0.0
        nodeA=[node_list[0],0.0,0.0]
        nodeA[1]=node_lonlat_dict[nodeA[0]][1]
        nodeA[2]=node_lonlat_dict[nodeA[0]][2]
        node_list_len=len(node_list)
        for i in range(1,node_list_len):
            nodeB=[node_list[i],0.0,0.0]
            nodeB[1]=node_lonlat_dict[nodeB[0]][1]
            nodeB[2]=node_lonlat_dict[nodeB[0]][2]
            #print(nodeA[0],nodeA[1],nodeA[2],nodeB[0],nodeB[1],nodeB[2])
            distance_res=haversine(nodeA[1],nodeA[2],nodeB[1],nodeB[2])
            #print(distance_res)
            length+=distance_res
            nodeA=copy.deepcopy(nodeB)
        return length
    except Exception as e:
        print('in calculate_road_length(): ',e)
        return 0
    

def renumber_node_id(origin_node_id,next_rank_id): 
    try:
        cur.execute("select node_new_id from node_numbers where node_id=%d"%(origin_node_id))
        result=cur.fetchall()
        if len(result)==0:
            cur.execute("insert into node_numbers values(%d,%d)"%(origin_node_id,next_rank_id))
        return cur.fetchall()[0][0]
    except:
        print('in renumber_node_id() : ',e)
        return origin_node_id
    
def write_latlon(fileObject,nodeid,nodeidNew):
    cur.execute("select id, st_x(geom) as lon, st_y(geom) as lat from qualified_nodes where id="+str(nodeid))
    result=cur.fetchall()
    longitude=result[0][1]
    latitude=result[0][2]
    fileObject.write(str(nodeidNew)+" "+str(longitude)+" "+str(latitude)+"\n")

In [ ]:
cur.execute("select distinct way_id from qualified_way_nodes")
res_qualified_way_id=cur.fetchall()
print("there are %d records in total."%(len(res_qualified_way_id)))

## 没有进行路网压缩,且没有POI

In [ ]:
%%time
map_file=open('../../map_maine/edge.txt','w')
latlon_file=open('../../map_maine/lonlat.txt','w')
#res_qualified_way_id=[[192046993,]]
count=0
renumbered_node=dict()
for row in res_qualified_way_id:
    if count%100==0:
        print('building the %d th road.'%(count))
    count+=1

    #start=time.perf_counter()
    qualified_way_id=row[0]
    cur.execute("select * from qualified_way_nodes where way_id=%d order by sequence_id"%(qualified_way_id))
    res_way_node=cur.fetchall() # [[way_id,node_id,sequenced_id],[],[]]
    #end=time.perf_counter()
    #print("time for select way nodes:",end-start)


    #print(res_way_node)
    sql_str="("
    for item in res_way_node:
        node=item[1]
        sql_str+=str(node)
        sql_str+=","
    sql_str=sql_str[:-1]
    sql_str+=")"
    cur.execute("select id, st_x(geom) as lon, st_y(geom) as lat from qualified_nodes where id in"+sql_str)
    node_lonlat_list=cur.fetchall() # [[id,lon,lat],[]]
    node_lonlat_dict=dict()
    for item in node_lonlat_list:# [id,lon,lat]
        node_lonlat_dict[item[0]]=item
 
    isFirst=True
    start_node=0
    start_node_renumber=0
    start_n_lonlat=[0,0,0]
    end_node=0
    end_n_lon_lat=[0,0,0]
    end_node_renumber=0
    for item in res_way_node: # [[way_id,node_id,sequenced_id],[],[]]
        if isFirst:
            start_node=item[1]
            start_n_lonlat=node_lonlat_dict[start_node]
            if start_node in renumbered_node:
                # 已经被重新编码过，在字典中获取之前的编码
                start_node_renumber=renumbered_node[start_node]
            else:
                # 没有被重新编码，就赋予新编码
                next_node_id=len(renumbered_node)
                renumbered_node[start_node]=next_node_id
                start_node_renumber=next_node_id
            str_write=str(start_node_renumber)+" "+str(start_n_lonlat[1])+" "+str(start_n_lonlat[2])+"\n"
            #print(str_write)
            latlon_file.write(str_write)
            isFirst=False
            continue
        end_node=item[1]
        end_n_lonlat=node_lonlat_dict[end_node]
        node_list=[start_node,end_node]
        #start=time.perf_counter()
        distance=haversine(start_n_lonlat[1],start_n_lonlat[2],end_n_lonlat[1],end_n_lonlat[2])
        #end=time.perf_counter()
        #print("time for computing distance:",end-start)

        #start=time.perf_counter()
        if end_node in renumbered_node:
            # 已经被重新编码过，在字典中获取之前的编码
            end_node_renumber=renumbered_node[end_node]
        else:
            # 没有被重新编码，就赋予新编码
            next_node_id=len(renumbered_node)
            renumbered_node[end_node]=next_node_id
            end_node_renumber=next_node_id
        #end=time.perf_counter()
        #print("time for renumber:",end-start)

        str_write=str(start_node_renumber)+' '+str(end_node_renumber)+' '+str(distance)+'\n'
        map_file.write(str_write)
        #print(str_write)
        
        latlon_str=str(end_node_renumber)+" "+str(end_n_lonlat[1])+" "+str(end_n_lonlat[2])+"\n"
        latlon_file.write(latlon_str)  
        #print(latlon_str)
        
        start_node=end_node
        start_n_lonlat=end_n_lonlat
        start_node_renumber=end_node_renumber


map_file.close()
latlon_file.close()
print("the total node number is %d"%(len(renumbered_node)))
        

## 路网压缩版本

In [ ]:
%%time
map_file=open('../../map_mississippi/edge.txt','w')
latlon_file=open('../../map_mississippi/lonlat.txt','w')
#res_qualified_way_id=[[192046993,]]
count=0
renumbered_node=dict()
for row in res_qualified_way_id:
    if count%100==0:
        print('building the %d th road.'%(count))
        
    count+=1
    qualified_way_id=row[0]

    #start=time.perf_counter()
    cur.execute("select * from qualified_way_nodes where way_id=%d order by sequence_id"%(qualified_way_id))
    res_way_node=cur.fetchall() # [[way_id,node_id,sequenced_id],[]]
    
    way_nodes_str="("
    for n in res_way_node:
        way_nodes_str+=str(n[1])
        way_nodes_str+=","
    way_nodes_str=way_nodes_str[:-1]
    way_nodes_str+=")"
    
    cur.execute("select id, st_x(geom) as lon, st_y(geom) as lat from qualified_nodes where id in "+way_nodes_str)
    res_node_lonlat=cur.fetchall() # [[id,lon,lat],[]]
    node_lonlat_dict=dict()
    for item in res_node_lonlat:
        node_lonlat_dict[item[0]]=item
        
    cur.execute("select node_id,freq from node_freq where node_id in "+way_nodes_str)
    res_node_freq=cur.fetchall() #[[node_id,freq],[]]
    node_freq_dict=dict()
    for item in res_node_freq:
        node_freq_dict[item[0]]=item[1]
        
    #end=time.perf_counter()
    #print('select node from qualified_way_nodes time consume: %f seconds.'%(end-start))
    res_len = len(res_way_node)
    
    road_length=0
    numofPOI=0
    road_list=[]
    curr_road_nodes=[]
    #print(res_way_node)
    end_node=0
    

    #global_start=time.perf_counter()
    #start=time.perf_counter()
    # 对每个node_id 进行处理,拆成多条road放到road list中

    is_first=True
    for node_res_item in res_way_node: # node_res_item [way_id,node_id,sequence_id]
        if is_first:
            is_first=False
            curr_road_nodes.append(node_res_item[1])
            continue
        # 查询当前node的出现次数(freq)，如果freq>=2即表示它在其他way中出现过，它是另一条路的交点，应该作为路径端点
        #print(node_res_item)
        curr_node_id=node_res_item[1]
        node_freq=node_freq_dict[curr_node_id]
        #print('node frequency: ',node_freq_res)
        if node_freq>=2: # 当前node应该是路径的终点,应该被分拆
            curr_road_nodes.append(curr_node_id)
            road=copy.deepcopy(curr_road_nodes)
            road_list.append(road)
            curr_road_nodes=[curr_node_id]
        else: # 当前node只是路径上的一个转折点
            curr_road_nodes.append(curr_node_id)

    #end=time.perf_counter()
    #print('split node to multiple road time consume: %f seconds'%(end-start))
    # 对road list里面的每条road进行计算
    for r in road_list: # road_list: [[node_id1,node_id2....],[],...]
        #start=time.perf_counter()
        distance_road=calculate_road_length(r,node_lonlat_dict)
        #end=time.perf_counter()
        #time_elapse=end-start
        #print('distance calculation consume time: %f seconds'%(time_elapse))

        #print(r[0],r[-1],distance_road,poiNum)
        # 将node的ID进行重新编码，从0开始
        # 首先查询该node是否已经重新编码
        start_node=int(r[0])
        end_node=int(r[-1])
        renumber_start_node=start_node
        renumber_end_node=end_node

        if start_node in renumbered_node:
            # 已经被重新编码过，在字典中获取之前的编码
            renumber_start_node=renumbered_node[start_node]
        else:
            # 没有被重新编码，就赋予新编码
            next_node_id=len(renumbered_node)
            renumbered_node[start_node]=next_node_id
            renumber_start_node=next_node_id

        if end_node in renumbered_node:
            # 已经被重新编码过，在字典中获取之前的编码
            renumber_end_node=renumbered_node[end_node]
        else:
            # 没有被重新编码，就赋予新编码
            next_node_id=len(renumbered_node)
            renumbered_node[end_node]=next_node_id
            renumber_end_node=next_node_id

        map_file.write(str(renumber_start_node)+' '+str(renumber_end_node)+' '+str(distance_road)+'\n')
        latlon_file.write(str(renumber_start_node)+' '+str(node_lonlat_dict[start_node][1])+' '+str(node_lonlat_dict[start_node][2])+'\n')
        latlon_file.write(str(renumber_end_node)+' '+str(node_lonlat_dict[end_node][1])+' '+str(node_lonlat_dict[end_node][2])+'\n')
    #global_end=time.perf_counter()
    #print('building road time consume: %f seconds'%(global_end-global_start))
map_file.close()
latlon_file.close()
print("the total node number is %d"%(len(renumbered_node)))

# 优化方法

* 尽可能在数据库内部进行运算，将计算结果保存为新表，不要逐个取出数据再运算
* 执行多条sql后再commit，不要每执行一条查询就commit一次
* 使用in而不是多个条件or
* 多个筛选条件放在一起使用where子句带in或者exist解决
* 多条数据需要查询的时候，先全部查询出来，保存在一个dict里面。而不是每条都发起一个query
* 几万到十几万条的数据使用dict保存而不是通过数据库查询

# playground

In [ ]:
conn.rollback()